# ZTF Sparse photometry for Asteroids

**IMPORTANT: In order to run this notebook you must firstly run the Asteroid_ZTF_FP_Request.ipynb**

This notebook uses data from ZTF Forced Photometry Service, whose manual can be found here:
<https://irsa.ipac.caltech.edu/data/ZTF/docs/ztf_forced_photometry.pdf>

### Useful packages

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import math
from jdcal import MJD_0, jd2gcal
import io
from IPython.display import clear_output
from scipy.optimize import curve_fit
from astroquery.jplhorizons import Horizons
import astrobase_custom as abc
from os.path import exists

## Asteroid Info and Datasets

In [ ]:
#Write only the id of asteroid:
asteroid_id=4760

#Write the full name of asteroid:
asteroid_name='4760 Jia-xiang (1981 GN1)'

#Import the ZTF file corresponds to the above asteroid:
ZTF_file='ZTF/MOST/'+str(asteroid_id)+'_ZTF.tbl'

#Would you like to save the figures? (True or False)
SaveFigs=True

#Would you like to save the final files? (True or False)
SaveFiles=True

### Read output file from MOST

The search of ZTF images, that contain the selected asteroid, was performed using the Moving Object Search Tool (MOST). This tool is available at: <https://irsa.ipac.caltech.edu/applications/MOST/>
The downloaded file should have name; <asteroid_id>_ZTF.tbl and be placed in; MOST folder.

In [ ]:
if exists(ZTF_file):
    data=pd.read_table(ZTF_file, header=None, skiprows=[14,15,16], comment='\\',  delim_whitespace=True, names=['Image_ID', 'date_obs', 'time_obs', 'mjd_obs', 'ra_obj', 'dec_obj', 'sun_dist', 'geo_dist', 'dist_ctr', 'phase', 'vmag', 'image_url', 'postcard_url', 'region_file'])
    dataR=data[data['Image_ID'].str.contains("_zr_")]
else:
    print("No ZTF data file! \n You must follow the above instructions.")

dataR

In [ ]:
fig1 = plt.figure(figsize=[15,4])

cm = plt.cm = plt.colormaps['plasma']

plt.scatter(dataR['phase'], dataR['vmag'], c=dataR['mjd_obs'], s=10, cmap=cm)

plt.title("Phase curve of " + asteroid_name + ' from ZTF (r filter) data')
plt.xlabel("α (deg)")
plt.ylabel("V Magnitude")
plt.xlim(0,30)
plt.ylim(int(min(dataR['vmag'])),math.ceil(max(dataR['vmag'])))
plt.colorbar(label='MJD', format=mtick.FuncFormatter(lambda x, pos: '%.0f'%x))
plt.clim(int(min(dataR['mjd_obs'])/100)*100,math.ceil(max(dataR['mjd_obs'])/100)*100)
plt.gca().invert_yaxis()

if SaveFigs: plt.savefig('ZTF/Figures/'+str(asteroid_id)+'_fig_1.png', bbox_inches='tight')

### Data from ZTF Forced Photometry Service
**Check the status or history of your submissions**
Now you have to check your email. You will receive an email from ZTF Forced Photometry service for each submission.

You can check the status or history of your submissions using the utility:
<https://ztfweb.ipac.caltech.edu/cgi-bin/getForcedPhotometryRequests.cgi>
The username, password for initiating this service is also: **ztffps**, **dontgocrazy!**

Due to possible excessive database loads, the job queue is refreshed every hour. It is refreshed on the hour according to wall clock.
Go to the history of your submission copy the cells of your last submissions and paste them to a file with name <asteroid_id>_ZTF_FP.txt in; ForcedPhot folder.

**Note: Be aware that you have to wait 1 hour in order to see your full status of your history!**

Then the following commands will read the file and retrieve the data.

**Exit and warning codes (the "procstatus" column)**
Overall exit and warning codes from execution of your request are provided in the log file that accompanies the output table. Processing status codes are also provided on a per-epoch basis. This is the procstatus column in the output table. These codes are defined below.
- 0 => Successful execution
- 56 => One or more epochs have photometry measurements that may be impacted by bad (including NaN'd) pixels
- 57 => One or more epochs had no reference image catalog source falling with 5 arcsec
- 58 => One or more epochs had a reference image PSF-catalog that does not exist in the archive
- 59 => One or more epochs may have suspect photometric uncertainties due to early creation date of difference image in production
- 60 => One or more epochs had upsampled diff-image PSF dimensions that were not odd integers
- 61 => One or more epochs had diff-image cutouts that were off the image or too close to an edge
- 62 => Requested start JD was before official survey start date [3/17/18] and was reset to 2018-03-17T00:00:00.0 UT
- 63 => No records (epochs) returned by database query
- 64 => Catastrophic error (see log output)
- 65 => Requested end JD is before official survey start date [3/17/18]
- 255 => Database connection or query execution error (see log output)

In [ ]:
results=pd.read_table('ZTF/ForcedPhot/'+str(asteroid_id)+'_ZTF_FP.txt', header=None, comment='#',  delim_whitespace=True, names=['reqId', 'ra', 'dec', 'startJD', 'endJD', 'created_date', 'created_time', 'started_date', 'started_time', 'ended_date', 'ended_time', 'machine', 'exitcode', 'lightcurve'])
results

In [ ]:
#Run it only ones, if you have select the option of saving files.
if exists("ZTF/ForcedPhot/"+str(asteroid_id)+"_FP_data.csv"):
    print("ZTF Forced Photometry data file exists!")
    finaldata=pd.read_csv("ZTF/ForcedPhot/"+str(asteroid_id)+"_FP_data.csv")

elif exists('ZTF/ForcedPhot/'+str(asteroid_id)+'_ZTF_FP.txt'):
    c=0
    for i in results.index:
        clear_output(wait=True)
        print("Progress "+str(i)+"/"+str(max(results.index)))
        x=requests.get('https://ztfweb.ipac.caltech.edu'+results["lightcurve"][i], auth=('ztffps', 'dontgocrazy!'))
        if c==0 and x.text!='':
            finaldata=pd.read_table(io.StringIO(x.text), sep=' ', comment='#', skipinitialspace=True)
            finaldata.columns = finaldata.columns.str.replace('[,]', '', regex=True)
            c=1
        elif x.text!='':
            singledata=pd.read_table(io.StringIO(x.text), sep=' ', comment='#', skipinitialspace=True)
            singledata.columns = finaldata.columns.str.replace('[,]', '', regex=True)
            finaldata=pd.concat([finaldata,singledata])
            del singledata

    clear_output(wait=True)
    print("Done!")
    finaldata.sort_values('jd')
    if SaveFiles: finaldata.to_csv("ZTF/ForcedPhot/"+str(asteroid_id)+"_FP_data.csv", index=False)
else:
    print("No ZTF Forced Photometry file exists!")

finaldata

In [ ]:
print("Exit codes:")
finaldata['procstatus'].value_counts()

### Remove data with unkown flux and zero point
Remove rows with NaN values of 'forcediffimflux' and 'zpdiff'

In [ ]:
cleardata=finaldata.dropna(subset=['forcediffimflux', 'zpdiff']).sort_values(by='jd',ignore_index=True)

In [ ]:
fig2 = plt.figure(figsize=[15,2])

plt.title("ZTF r-filter Data for " + asteroid_name)

plt.errorbar(cleardata['jd'], cleardata['forcediffimfluxap'], cleardata['forcediffimfluxuncap'], markersize=4, fmt='.',label='Curve-of-growth-corrected aperture photometry')
plt.errorbar(cleardata['jd'], cleardata['forcediffimflux'], cleardata['forcediffimfluxunc'], markersize=4, fmt='.', label='PSF-fit photometry')

plt.ylabel("Differential Flux")
plt.xlabel("JD")
plt.gca().xaxis.set_major_formatter(mtick.FuncFormatter(lambda x, pos: '%.2f'%x))

plt.legend()

if SaveFigs: plt.savefig('ZTF/Figures/'+str(asteroid_id)+'_fig_2.png', bbox_inches='tight')

### Validating photometric uncertainties and possible rescaling

In [ ]:
fig3 = plt.figure(figsize=[15,2])

plt.title("ZTF r-filter Data for " + asteroid_name)
plt.scatter(cleardata['forcediffimflux'], cleardata['forcediffimfluxunc'], s=4,label='Curve-of-growth-corrected aperture photometry')
plt.scatter(cleardata['forcediffimflux'], cleardata['forcediffimfluxunc'], s=4, label='PSF-fit photometry')
plt.ylabel("Differential Flux Error")
plt.xlabel("Differential Flux")
plt.xlim([0, 15000])

plt.legend()

if SaveFigs: plt.savefig('ZTF/Figures/'+str(asteroid_id)+'_fig_3.png', bbox_inches='tight')

In [ ]:
fig4 = plt.figure(figsize=[15,2])

plt.title("ZTF r-filter Data for " + asteroid_name)
[counts, bins, bars]=plt.hist(cleardata['forcediffimchisq'], bins=200)
plt.ylabel("Measurements")
plt.xlabel("χ$^2$")
plt.xlim([0,40])

plt.vlines(1,0,max(counts)+5, color='g', linestyles='--', label=r'$\chi^2=1$')

plt.vlines(np.average(cleardata['forcediffimchisq']),0,max(counts)+5, 'r', linestyles='--', label=r'$\langle \chi^2 \rangle = $' + "{:.3f}".format(np.average(cleardata['forcediffimchisq'])))

plt.legend()
if SaveFigs: plt.savefig('ZTF/Figures/'+str(asteroid_id)+'_fig_4.png', bbox_inches='tight')

In [ ]:
if abs(np.average(cleardata['forcediffimchisq'])-1)>0.1:
    print('The flux errors need correction.')
    pd.options.mode.chained_assignment = None  # default='warn'
    cleardata['forcediffimfluxunc']=cleardata['forcediffimfluxunc']*np.sqrt(np.average(cleardata['forcediffimchisq']))
    pd.options.mode.chained_assignment = 'warn'
    print('Correction is done!')
else:
    print('The flux errors do not need correction.')

### Generating differential-photometry lightcurves with non-detections
It is assumed you have corrected your differential fluxes ('forcediffimflux') for any non-zero baseline and validated and possibly rescaled the uncertainties ('forcediffimfluxunc'). If the uncertainties are over- or underestimated, your significance levels and hence derived upper limits will be wrong. I.e., “5-sigma” only has meaning if “sigma” is correct, that is, it captures all noise sources that may have corrupted the measurements.
The pseudo-code for obtaining calibrated magnitudes is below. This uses the following two parameters:

SNT – the signal-to-noise threshold for declaring a measurement a “non-detection” so it can be assigned an upper-limit.
SNU – the actual signal-to-noise ratio to use when computing a “SNU-sigma” upper-limit.

It is recommended SNT = 3 and SNU = 5. For details on why these were picked, see:
<http://web.ipac.caltech.edu/staff/fmasci/home/mystats/UpperLimits_FM2011.pdf>


In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'
#Create new columns
cleardata['mag']=''
cleardata['magerr']=''
cleardata['detection']=''

SNT=3
SNU=5
for i in cleardata.index:
    if cleardata['forcediffimflux'][i]/cleardata['forcediffimfluxunc'][i] > SNT :
        # we have a “confident” detection, compute and plot mag with error bar:
        cleardata['mag'][i]=cleardata['zpdiff'][i]-2.5*np.log10(cleardata['forcediffimflux'][i])
        cleardata['magerr'][i]=1.0857* cleardata['forcediffimfluxunc'][i]/cleardata['forcediffimflux'][i]
        cleardata['detection'][i]=True
    else:
        # compute flux upper limit and plot as arrow:
        cleardata['mag'][i]=cleardata['zpdiff'][i]-2.5*np.log10(SNU*cleardata['forcediffimfluxunc'][i])
        cleardata['magerr'][i]=np.nan
        cleardata['detection'][i]=False

pd.options.mode.chained_assignment = 'warn'

In [ ]:
fig5 = plt.figure(figsize=[15,2])

plt.title("ZTF r-filter Data for " + asteroid_name)

plt.errorbar(cleardata['jd'][cleardata['detection']==True], cleardata['mag'][cleardata['detection']==True], cleardata['magerr'][cleardata['detection']==True], fmt='.', label='Confident detections')

plt.scatter(cleardata['jd'][cleardata['detection']==False], cleardata['mag'][cleardata['detection']==False], s=8, color='r', marker="^", label="False detections")

plt.ylabel("Calibrated Magnitudes")
plt.xlabel("JD")
plt.gca().xaxis.set_major_formatter(mtick.FuncFormatter(lambda x, pos: '%.2f'%x))

plt.legend()

if SaveFigs: plt.savefig('ZTF/Figures/'+str(asteroid_id)+'_fig_5.png', bbox_inches='tight')

### Remove false detections

In [ ]:
cleardata=cleardata.drop(cleardata[cleardata.detection == False].index)
cleardata=cleardata.drop(columns=['detection'])

### Import alpha angle and distances
α: Alpha phase angle, Solar-Target-Earth angle (S-T-O)
r: Heliocentric distance of the asteroid
Delta: Geocentric distance of the asteroid

In [ ]:
#For some reason, Horizons service is not working for a list of more than 77 julian dates.
#Set a limit for each Horizons call.
lim=74 #It must be <=75.

if len(cleardata['jd'])>lim:
    n=math.ceil(len(cleardata['jd'])/lim)
    start=0
    end=lim
    for i in range(n):
        obj = Horizons(id=str(asteroid_id), location='g@399', epochs=cleardata['jd'][start:end+1].to_list())
        if i==0:
            alphas=obj.ephemerides()["alpha_true"].tolist()
            r=obj.ephemerides()["r"].tolist()
            delta=obj.ephemerides()["delta"].tolist()
            lt=obj.ephemerides()["lighttime"].tolist()
        else:
            alphas=alphas + obj.ephemerides()["alpha_true"].tolist()
            r=r+obj.ephemerides()["r"].tolist()
            delta=delta + obj.ephemerides()["delta"].tolist()
            lt=lt+obj.ephemerides()["lighttime"].tolist()

        start=end+1
        if len(cleardata['jd'])>(end+lim):
            end =end+lim
        else:
            end= len(cleardata['jd'])-1

else:
    obj = Horizons(id=str(asteroid_id), location='g@399', epochs=cleardata['jd'].to_list())
    alphas = obj.ephemerides()["alpha_true"].tolist()
    r=obj.ephemerides()["r"].tolist()
    delta=obj.ephemerides()["delta"].tolist()
    lt=obj.ephemerides()["lighttime"].tolist()

cleardata['alpha']=alphas
cleardata['r']=r
cleardata['delta']=delta
cleardata['jd_c']=cleardata['jd']-np.array(lt)/60/24

### Calculate reduced magnitudes and find outliers

In [ ]:
cleardata['Rmag']=cleardata['mag']-5*np.log10(cleardata['r'])-5*np.log10(cleardata['delta'])

[JDc, Dmc, DmErrc] = abc.sigclip_magseries(np.array(cleardata['jd'], dtype=float), np.array(cleardata['Rmag'], dtype=float), np.array(cleardata['magerr'], dtype=float), sigclip=[2.8, 2.8], iterative=True, niterations=10000, meanormedian='mean',magsarefluxes=False)

pd.options.mode.chained_assignment = None  # default='warn'
cleardata['outlier'] = [False] * len(cleardata.index)

for i in cleardata.index:
    if cleardata['jd'][i] not in JDc: cleardata['outlier'][i] =  True

pd.options.mode.chained_assignment = 'warn'

In [ ]:
fig6 = plt.figure(figsize=[15,2])

plt.title("ZTF r-filter Data for " + asteroid_name)

plt.errorbar(cleardata['jd'], cleardata['Rmag'], cleardata['magerr'], fmt='.', zorder=1, label='Data')
plt.scatter(cleardata['jd'][cleardata.outlier==True], cleardata['Rmag'][cleardata.outlier==True], marker='x', color='r', zorder=2, label='Outliers')

plt.ylabel("Reduced Magnitudes")
plt.xlabel("JD")

plt. legend()

plt.gca().xaxis.set_major_formatter(mtick.FuncFormatter(lambda x, pos: '%.2f'%x))

if SaveFigs: plt.savefig('ZTF/Figures/'+str(asteroid_id)+'_fig_6.png', bbox_inches='tight')

### Define the Apparitions

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

cleardata['apparition']=""
T0=cleardata["jd"].iloc[0]
app=1
for i in cleardata.index:
    if (cleardata['jd'][i]-T0)<365:
        cleardata['apparition'][i]=app
    else:
        T0=cleardata['jd'][i]
        app+=1
        cleardata['apparition'][i]=app

pd.options.mode.chained_assignment = 'warn'

print("Total number of apparitions: "+str(app))

In [ ]:
fig7 = plt.figure(figsize=[8,4])

plt.title("Number of ZTF r-filter Data per Apparition for " + asteroid_name)

plt.bar(*np.unique(cleardata['apparition'], return_counts=True))
plt.ylabel("Measurements")
plt.xlabel("Apparitions")
plt.xticks(np.arange(1, app+1, 1.0))
plt.gca().xaxis.set_major_formatter(mtick.FuncFormatter(lambda x, pos: '%.0f'%x))

if SaveFigs: plt.savefig('ZTF/Figures/'+str(asteroid_id)+'_fig_7.png', bbox_inches='tight')

 ### Asteroid phase curve

The brightness vs. solar phase angle relation can be fitted with a simple relation:
$$f(\alpha)=A_0 \cdot e^{- \alpha / D} - k \cdot \alpha +1$$
where $α$ is the solar phase angle(the Sun - asteroid - Earth angle) and $A_0$, $D$ and $k$ are free parameters.

In cases with a small number of data points (≲30) or a poor sampling of low phase angles ($α \gtrsim 10^\o$) the fitting sometimes fails. For these cases we only use the linear part of the phase function for outlier rejection (i.e., we set $A_0=0$).


In [ ]:
# Phase curve function
def phasecurve(x, A_0, D, k):
    return A_0*np.exp(-1*(x * np.pi / 360)/D) - k*x +1

In [ ]:
fig8, axs = plt.subplots(app, 1, figsize=[10,3*app])
A_0=[]
D=[]
k=[]

for i in range(4, app+1):
    # curve fit
    [popt, pcov] = curve_fit(phasecurve, cleardata['alpha'][(cleardata.apparition==i) & (cleardata.outlier==False)].tolist(), cleardata['Rmag'][(cleardata.apparition==i) & (cleardata.outlier==False)].tolist(), sigma=cleardata['magerr'][(cleardata.apparition==i) & (cleardata.outlier==False)].tolist(), p0=[1., 1., 1.])

    print("\nApparition " + str(i) + ":")

    print("A_0 =", popt[0], "±", pcov[0, 0] ** 0.5)
    print("D =", popt[1], "±", pcov[1, 1] ** 0.5)
    print("k =", popt[2], "±", pcov[2, 2] ** 0.5)

    # summarize the parameter values
    A_0.append([popt[0], pcov[0, 0] ** 0.5])
    D.append([popt[1], pcov[1, 1] ** 0.5])
    k.append([popt[2], pcov[2, 2] ** 0.5])

    axs[i-1].scatter(cleardata['alpha'][cleardata.apparition==i], cleardata['Rmag'][cleardata.apparition==i], s=10, color='k')
    axs[i-1].scatter(cleardata['alpha'][(cleardata.apparition==i) & (cleardata.outlier==True)], cleardata['Rmag'][(cleardata.apparition==i) & (cleardata.outlier==True)], color='r',  marker='x')



    x_line = np.arange(0, 40, 1)
    y_line = phasecurve(x_line, popt[0], popt[1], popt[2])
    axs[i-1].plot(x_line, y_line, '--', color='red')


    axs[i-1].title.set_text("Apparition " + str(i) + ": " + str(jd2gcal(min(cleardata['jd'][cleardata.apparition==i].tolist()),0)[1]) + "." + str(jd2gcal(min(cleardata['jd'][cleardata.apparition==i]),0)[0]) + " - " + str(jd2gcal(max(cleardata['jd'][cleardata.apparition==i]),0)[1]) + "." + str(jd2gcal(max(cleardata['jd'][cleardata.apparition==i]),0)[0]))

    axs[i-1].set_xlabel("α (deg)")
    axs[i-1].set_ylabel("Reduced Magnitude")

    axs[i-1].set_xlim(0, 30)

    axs[i-1].invert_yaxis()
    axs[i-1].text(0.01,0.05, '$f(α)=A_0 \cdot e^{- α / D} - k \cdot α +1$' + "\n$Α_0$ =" + "{:.5f}".format(popt[0]) + " ± " + "{:.5f}".format(pcov[0, 0] ** 0.5) + "\nD =" + "{:.5f}".format(popt[1]) + " ± " + "{:.5f}".format(pcov[1, 1] ** 0.5) + "\nk =" + "{:.5f}".format(popt[2]) + " ± " + "{:.5f}".format(pcov[2, 2] ** 0.5),  transform=axs[i-1].transAxes)
plt.tight_layout()

if SaveFigs: plt.savefig('ZTF/Figures/'+str(asteroid_id)+'_fig_8.png', bbox_inches='tight')

### Convert to relative fluxes

We transform into fluxes utilising the Pogson equation and, for convenience, setting the zero magnitude to 15.

In [ ]:
cleardata['rel_flux']=10**(-0.4*(cleardata['Rmag']-15))
cleardata['rel_flux_err']=abs(0.4*np.log(10)*(-1*10**(-0.4*(cleardata['Rmag']-15)))*cleardata['magerr'])
pd.options.mode.chained_assignment = None  # default='warn'


#Align data
medians=0
for i in range(1,app+1):
    medians+=np.median(cleardata['rel_flux'][(cleardata.apparition==i) & (cleardata.outlier==False)])

avrg_medians=medians/app

for i in range(1,app+1):
    cleardata['rel_flux'][cleardata.apparition==i]=cleardata['rel_flux'][cleardata.apparition==i]+(avrg_medians-np.median(cleardata['rel_flux'][(cleardata.apparition==i)& (cleardata.outlier==False)]))


#[A, RFc, RFEc] = abc.sigclip_magseries(np.array(cleardata.sort_values(by=['alpha'], ascending=True)['alpha'], dtype=float), np.array(cleardata.sort_values(by=['alpha'], ascending=True)['rel_flux'], dtype=float), np.array(cleardata.sort_values(by=['alpha'], ascending=True)['rel_flux_err'], dtype=float), sigclip=[3, 3], iterative=True, niterations=10000, meanormedian='mean',magsarefluxes=True)


#for i in cleardata.index:
#    if cleardata['alpha'][i] not in A: cleardata['outlier'][i] =  True

pd.options.mode.chained_assignment = 'warn'

In [ ]:
fig9 = plt.figure(figsize=[15,2])

plt.title("ZTF r-filter Data for " + asteroid_name)

plt.errorbar(cleardata['jd'], cleardata['rel_flux'], cleardata['rel_flux_err'], fmt='.', zorder=1, label='Data')
plt.scatter(cleardata['jd'][cleardata.outlier==True], cleardata['rel_flux'][cleardata.outlier==True], marker='x', color='r', zorder=2, label='Outliers')

plt.ylabel("Relative Flux")
plt.xlabel("JD")

plt. legend()

plt.gca().xaxis.set_major_formatter(mtick.FuncFormatter(lambda x, pos: '%.2f'%x))

if SaveFigs: plt.savefig('ZTF/Figures/'+str(asteroid_id)+'_fig_9.png', bbox_inches='tight')

In [ ]:
# curve fit
[popt, pcov] = curve_fit(phasecurve, cleardata['alpha'][cleardata.outlier==False].tolist(), cleardata['rel_flux'][cleardata.outlier==False].tolist(), sigma=cleardata['magerr'][cleardata.outlier==False].tolist(), p0=[1., 1., 1.])

print("A_0 =", popt[0], "±", pcov[0, 0] ** 0.5)
print("D =", popt[1], "±", pcov[1, 1] ** 0.5)
print("k =", popt[0], "±", pcov[2, 2] ** 0.5)

In [ ]:
cleardata['residuals']=cleardata['rel_flux'] - phasecurve(cleardata['alpha'], popt[0], popt[1], popt[2])
[A, RFc, RFEc] = abc.sigclip_magseries(np.array(cleardata.sort_values(by=['alpha'], ascending=True)['alpha'], dtype=float), np.array(cleardata.sort_values(by=['alpha'], ascending=True)['residuals'], dtype=float), np.array(cleardata.sort_values(by=['alpha'], ascending=True)['rel_flux_err'], dtype=float), sigclip=[5, 5], iterative=True, niterations=10000, meanormedian='mean',magsarefluxes=True)

pd.options.mode.chained_assignment = None  # default='warn'
for i in cleardata.index:
    if cleardata['alpha'][i] not in A: cleardata['outlier'][i] =  True
    if cleardata['residuals'][i]>0.475: cleardata['outlier'][i] =  True #0.7 mag
    if cleardata['magerr'][i]>0.15: cleardata['outlier'][i] = True

pd.options.mode.chained_assignment = 'warn'

In [ ]:
fig10, axs = plt.subplots(app, 1, figsize=[10,3*app])

for i in range(1, app+1):


    axs[i-1].scatter(cleardata['alpha'][cleardata.apparition==i], cleardata['rel_flux'][cleardata.apparition==i], s=10, color='k')
    axs[i-1].scatter(cleardata['alpha'][(cleardata.apparition==i) & (cleardata.outlier==True)], cleardata['rel_flux'][(cleardata.apparition==i) & (cleardata.outlier==True)], color='r',  marker='x')



    x_line = np.arange(0, 40, 1)
    y_line = phasecurve(x_line, popt[0], popt[1], popt[2])
    axs[i-1].plot(x_line, y_line, '--', color='red')


    axs[i-1].title.set_text("Apparition " + str(i) + ": " + str(jd2gcal(min(cleardata['jd'][cleardata.apparition==i].tolist()),0)[1]) + "." + str(jd2gcal(min(cleardata['jd'][cleardata.apparition==i]),0)[0]) + " - " + str(jd2gcal(max(cleardata['jd'][cleardata.apparition==i]),0)[1]) + "." + str(jd2gcal(max(cleardata['jd'][cleardata.apparition==i]),0)[0]))

    axs[i-1].set_xlabel("α (deg)")
    axs[i-1].set_ylabel("Relative Flux")

    axs[i-1].set_xlim(0, 30)

    axs[i-1].text(0.01,0.05, '$f(α)=A_0 \cdot e^{- α / D} - k \cdot α +1$' + "\n$Α_0$ =" + "{:.5f}".format(popt[0]) + " ± " + "{:.5f}".format(pcov[0, 0] ** 0.5) + "\nD =" + "{:.5f}".format(popt[1]) + " ± " + "{:.5f}".format(pcov[1, 1] ** 0.5) + "\nk =" + "{:.5f}".format(popt[2]) + " ± " + "{:.5f}".format(pcov[2, 2] ** 0.5),  transform=axs[i-1].transAxes)
plt.tight_layout()

if SaveFigs: plt.savefig('ZTF/Figures/'+str(asteroid_id)+'_fig_10.png', bbox_inches='tight')

### Save Dataframe to a pickle file
This file it will be used in order to define the weight with all sparse data.

In [ ]:
SavedData=cleardata[['jd', 'rel_flux', 'rel_flux_err', 'alpha']][cleardata.outlier==False]
pd.options.mode.chained_assignment = None  # default='warn'
SavedData['observatory']='ZTF'
SavedData['apparition']=cleardata['apparition'][cleardata.outlier==False]
SavedData['rel_flux']=SavedData['rel_flux']-np.average(SavedData['rel_flux'])+1
pd.options.mode.chained_assignment = 'warn'
if SaveFiles: SavedData.to_pickle("ZTF/Pickles/"+str(asteroid_id)+".pkl")

# The end